In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch import nn
import torchaudio
from torchaudio import datasets
import torchtext
from torchaudio.utils import download_asset

import os
import time
from tqdm import tqdm
import IPython.display as ipd

In [ ]:
SAMPLE_WAV_SPEECH_PATH = download_asset("tutorial-assets/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav")

100%|██████████| 106k/106k [00:00<00:00, 20.4MB/s]


In [ ]:
wav, sr = torchaudio.load(SAMPLE_WAV_SPEECH_PATH)

In [ ]:
f_mfcc = torchaudio.transforms.MFCC(sample_rate = 16000,
                                    n_mfcc = 80,
                                    melkwargs = {
                                        'f_min': 0,
                                        'f_max': 2048,
                                        'n_mels': 80,
                                        'win_length': 400,
                                        'hop_length': 160,
                                        'n_fft': 400
                                    })
f_mfcc(wav[:,:16000]).shape

In [2]:
train_dataset_pt = datasets.SPEECHCOMMANDS('./', 'speech_commands_v0.02', download = True, subset = 'training')
val_dataset_pt = datasets.SPEECHCOMMANDS('./', 'speech_commands_v0.02', download = True, subset = 'validation')
test_dataset_pt = datasets.SPEECHCOMMANDS('./', 'speech_commands_v0.02', download = True, subset = 'testing')

100%|██████████| 2.26G/2.26G [00:23<00:00, 103MB/s]


In [14]:
import random

def pad_truncate(wav, max_length = 16000, pad_value = 0):
    wav_length = len(wav)
    if wav_length < max_length:
        pad = torchtext.transforms.PadTransform(max_length, 0)
        wav = pad(wav)
    return wav

def time_shift(wav, shift, sr = 16000, max_length = 16000):
    """

    shift: float
        Unit: Seconds
    """
    wav = torch.roll(wav, int(shift*sr))
    return wav[:, :max_length]

## Add Noise

def normalzieNoise(wav, noise, max_length = 16000):
    len_wav = wav.shape[1]
    len_noise = noise.shape[1]
    if len_wav > len_noise:
        buf = torch.zeros_like(wav)
        start_point = int((len_wav - len_noise)*random.uniform(0, 1))
        end_point = start_point + len_noise
        buf[:, start_point: end_point] = noise
        noise = buf
    elif len_wav < len_noise:
        start_point = int((len_noise - len_wav)*random.uniform(0, 1))
        end_point = start_point + len_wav
        noise = noise[:, start_point: end_point]
    return noise[:, :max_length]

def randomNoise(noise_directory):
    listnoise = [f for f in os.listdir(noise_directory)
                 if f.endswith('.wav')]
    noise = random.choice(listnoise)
    noise, sr = torchaudio.load(os.path.join(noise_directory, noise))
    return noise

def addNoise(wav, noise):
    noise = normalzieNoise(wav, noise)
    addnsy = torchaudio.transforms.AddNoise()
    return addnsy(wav, noise, snr = torch.Tensor([random.uniform(0, 15)]))

class AddBGNoise(nn.Module):
    def __init__(self):
        super().__init__()
        self.NOISE_PATH = '/content/SpeechCommands/speech_commands_v0.02/_background_noise_'

    def forward(self, x):
        p = random.uniform(0, 1)
        if p <= 0.8:
            noise = randomNoise(self.NOISE_PATH)
            x = addNoise(x, noise)
        return x

class GSC_TrainAugment(nn.Module):
    def __init__(self, sr):
        super().__init__()
        self.resample = torchaudio.transforms.Resample(sr, int(sr*random.uniform(0.9, 1.1)))
        self.time_shift = lambda x: time_shift(x, random.uniform(-0.05, 0.05))
        self.pad_trunc = lambda x: pad_truncate(x, sr)
        self.add_noise = AddBGNoise()
        self.mfcc = torchaudio.transforms.MFCC(sample_rate = 16000,
                                    n_mfcc = 80,
                                    melkwargs = {
                                        'f_min': 0,
                                        'f_max': 2048,
                                        'n_mels': 80,
                                        'win_length': 400,
                                        'hop_length': 160,
                                        'n_fft': 400
                                    })
        #self.specaugment = torchaudio.transforms.SpecAugment(n_time_masks = 2,
        #                              time_mask_param = 20,
        #                              n_freq_masks = 2,
        #                              freq_mask_param = 40)

    def forward(self, x):
        x = self.resample(x)
        x = self.time_shift(x)
        x = self.pad_trunc(x)
        x = self.add_noise(x)
        x = self.mfcc(x)
        #x = self.specaugment(x)
        return x

class GSC_TestAugment(nn.Module):
    def __init__(self, sr):
        super().__init__()
        self.pad_trunc = lambda x: pad_truncate(x, sr)
        self.mfcc = torchaudio.transforms.MFCC(sample_rate = 16000,
                                    n_mfcc = 80,
                                    melkwargs = {
                                        'f_min': 0,
                                        'f_max': 2048,
                                        'n_mels': 80,
                                        'win_length': 400,
                                        'hop_length': 160,
                                        'n_fft': 400
                                    })

    def forward(self, x):
        x = self.pad_trunc(x)
        x = self.mfcc(x)
        return x

train_transform = GSC_TrainAugment(16000)
test_transform = GSC_TestAugment(16000)

/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (80) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [5]:
from GSC import download_GSC

In [7]:
ZIP_MAP = download_GSC('https://drive.google.com/file/d/1hVXW_zSMNlJXxza_3Ojtgrq16q45dt3k/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-19CGvTGwZopQcT7VA0mxAGwvCjgx-cF/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-CdPCNkYcprSzlqGXgho_wICSjYPS55B/view?usp=drive_link',
                       '/content/tf_mdata',
                       end = '.zip')
CSV_MAP = download_GSC('https://drive.google.com/file/d/1-EZJqm6VLTLkCFefpqBNX25vre9r9HoX/view?usp=drive_link',
                       'https://drive.google.com/file/d/1ZEJLFxuevd44xwFIVfJv2F7S2YHUPz9k/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-1ouA4TZGbwlpemo-1dScGlHCQjTwQDr/view?usp=drive_link',
                       '/content/tf_mdata',
                       end = '.csv')

Downloading...
From (original): https://drive.google.com/uc?id=1hVXW_zSMNlJXxza_3Ojtgrq16q45dt3k
From (redirected): https://drive.google.com/uc?id=1hVXW_zSMNlJXxza_3Ojtgrq16q45dt3k&confirm=t&uuid=7cdbe6e2-4831-48c8-a9e6-fdf5ff75534c
To: /content/tf_mdata/train.zip
100%|██████████| 1.98G/1.98G [00:17<00:00, 110MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-19CGvTGwZopQcT7VA0mxAGwvCjgx-cF
From (redirected): https://drive.google.com/uc?id=1-19CGvTGwZopQcT7VA0mxAGwvCjgx-cF&confirm=t&uuid=34df9204-c61e-43f8-9bd2-8219304e4f21
To: /content/tf_mdata/val.zip
100%|██████████| 234M/234M [00:01<00:00, 191MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-CdPCNkYcprSzlqGXgho_wICSjYPS55B
From (redirected): https://drive.google.com/uc?id=1-CdPCNkYcprSzlqGXgho_wICSjYPS55B&confirm=t&uuid=35eab0e5-0cd3-40a9-81c2-a19748978968
To: /content/tf_mdata/test.zip
100%|██████████| 113M/113M [00:02<00:00, 49.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-E

In [11]:
def GSC_preprocessing(csv_file, root, output_directory, num_classes = 12, transform = None,
                       mul_factor = 1, set = 'train', csv_file_name = 'analysised_spec.csv'):
    """
    Preprocessing for each dataset

    mul_factor: increasing the number of data samples by mul_factor times.
    """
    out_df = {
        'link': [],
        'label': [],
    }
    data_df = pd.read_csv(csv_file)

    for idx in range(mul_factor):
        # def f(ix, ex):
        for ix in tqdm(range(len(data_df))):
            #if ix%1000 == 0:
            #    print(f'{ix}/{len(data_df)}')
            row = data_df.iloc[ix]
            label = row['label']
            wav = np.load(os.path.join(root, row['link']))['arr_0']

            fname = f'{set}_{label}_{ix}_{idx}.npz'
            #row = {
            #    'link': os.path.join(output_directory, f'label_{label}', fname),
            #    'label': label,
            #    'set': set
            #}
            out_df['link'].append(os.path.join(set, fname))
            out_df['label'].append(label)

            if os.path.exists(os.path.join(output_directory, fname)):
                continue

            wav = torch.from_numpy(wav).unsqueeze(0)

            if transform:
                wav = transform(wav.float())

            #out_df = out_df._append(row, ignore_index = True)
            np.savez_compressed(os.path.join(output_directory, fname), wav.squeeze(0).numpy())

        # Parallel(n_jobs = os.cpu_count())(delayed(f)(i, ex) for i, ex in tqdm(enumerate(dataset)))

    out_df = pd.DataFrame(out_df)
    out_df.to_csv(csv_file_name, index = False)

In [9]:
from GSC_zip import unzipzip, zipzip

In [10]:
unzipzip(ZIP_MAP['train'], '/content/train')
unzipzip(ZIP_MAP['val'], '/content/val')
unzipzip(ZIP_MAP['test'], '/content/test')

Extracted /content/tf_mdata/train.zip
Extracted /content/tf_mdata/val.zip
Extracted /content/tf_mdata/test.zip


In [ ]:
GSC_preprocessing(CSV_MAP['val'], '/content', '/content/val_3', transform = test_transform, set = 'val', csv_file_name = '/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC/val_12_mfcc.csv')

0/10102
1000/10102
2000/10102
3000/10102
4000/10102
5000/10102
6000/10102
7000/10102
8000/10102
9000/10102
10000/10102


In [ ]:
GSC_preprocessing(CSV_MAP['test'], '/content', '/content/test_3', transform = test_transform, set = 'test', csv_file_name = '/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC/test_12_mfcc.csv')

0/4890
1000/4890
2000/4890
3000/4890
4000/4890


In [17]:
GSC_preprocessing(CSV_MAP['train'], '/content', '/content/train_3', transform = train_transform, mul_factor = 2, set = 'train', csv_file_name = '/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC_nonspec/train_12_mfcc_noise_2.csv')

100%|██████████| 85511/85511 [42:48<00:00, 33.29it/s]


In [18]:
zipzip('/content/train_3', '/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC_nonspec/train_12_mfcc_noise_2.zip')

zipping...: 100%|██████████| 171022/171022 [05:09<00:00, 553.30it/s]


/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC_nonspec/train_12_mfcc_noise_2.zip created


In [ ]:
zipzip('/content/val_3', '/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC/val_12_mfcc.zip')
zipzip('/content/test_3', '/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC/test_12_mfcc.zip')
zipzip('/content/train_3', '/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC/train_12_mfcc.zip')


zipping...: 100%|██████████| 10102/10102 [00:27<00:00, 373.22it/s] 


/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC/val_12_mfcc.zip created



zipping...: 100%|██████████| 4890/4890 [00:08<00:00, 549.22it/s]


/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC/test_12_mfcc.zip created



zipping...:  54%|█████▍    | 46033/85511 [01:40<01:03, 625.34it/s]
 20%|██        | 396M/1.98G [31:39<2:06:19, 208kB/s]

zipping...: 100%|██████████| 85511/85511 [02:46<00:00, 515.11it/s]


/content/drive/MyDrive/GSC/Background_Noise/GSC_12_MFCC/train_12_mfcc.zip created


In [ ]:
wav_ = train_transform(wav)

ipd.Audio(wav_, rate = sr)

In [ ]:
resample = torchaudio.transforms.Resample(sr, int(sr*random.uniform(0.9, 1.1)))
wav1 = resample(wav)
resample = torchaudio.transforms.Resample(sr, int(sr*random.uniform(0.9, 1.1)))
wav2 = resample(wav)

In [ ]:
print(wav1.shape)
print(wav2.shape)

torch.Size([1, 54370])
torch.Size([1, 49878])


In [ ]:
wav.shape

torch.Size([1, 54400])

In [ ]:
ipd.Audio(wav, rate = sr)

In [ ]:
ipd.Audio(wav1, rate = sr)

In [ ]:
ipd.Audio(wav2, rate = sr)

In [ ]:
def time_shift(wav, shift, sr = 16000, max_length = 16000):
    """

    shift: float
        Unit: Seconds
    """
    wav = torch.roll(wav, int(shift*sr))
    return wav[:, :max_length]

## Add Noise

In [ ]:
tshift = lambda x: time_shift(x, random.uniform(-0.05, 0.05))
wav1 = tshift(wav)
tshift = lambda x: time_shift(x, random.uniform(-0.05, 0.05))
wav2 = tshift(wav)
print(wav1.shape)
print(wav2.shape)

torch.Size([1, 16000])
torch.Size([1, 16000])


In [ ]:
wav1

tensor([[ 0.0112,  0.0143,  0.0172,  ..., -0.1933, -0.1866, -0.1527]])

In [ ]:
wav2

tensor([[0.0045, 0.0045, 0.0045,  ..., 0.0059, 0.0148, 0.0242]])

In [ ]:
ipd.Audio(wav, rate = sr)

In [ ]:
ipd.Audio(wav1, rate = sr)

In [ ]:
ipd.Audio(wav2, rate = sr)